In [ ]:
import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot
from ufl import ds, dx, grad, inner

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

In [ ]:
msh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (4.0, 1.0)), n=(50, 50),
                            cell_type=mesh.CellType.triangle,)
V = fem.FunctionSpace(msh, ("Lagrange", 1))

In [ ]:
anode = mesh.locate_entities_boundary(msh, dim=1,
                                       marker=lambda x: np.isclose(x[0], 0.0))
cathode = mesh.locate_entities_boundary(msh, dim=1,
                                       marker=lambda x: np.isclose(x[0], 1.0))

In [ ]:
dofs1 = fem.locate_dofs_topological(V=V, entity_dim=1, entities=anode)
bc_anode = fem.dirichletbc(value=ScalarType(0), dofs=dofs1, V=V)

dofs2 = fem.locate_dofs_topological(V=V, entity_dim=1, entities=anode)
bc_cathode = fem.dirichletbc(value=ScalarType(0), dofs=dofs2, V=V)

In [ ]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
f = x[1] * x[1]
g = 1.0
a = inner(grad(u), grad(v)) * dx
L = inner(f, v) * dx + inner(g, v) * ds

In [ ]:
problem = fem.petsc.LinearProblem(a, L, bcs=[bc_anode, bc_cathode], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [ ]:
with io.XDMFFile(msh.comm, "out_poisson/poisson.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(uh)



In [ ]:
try:
    import pyvista
    cells, types, x = plot.create_vtk_mesh(V)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = uh.x.array.real
    grid.set_active_scalars("u")
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, show_edges=False)
    warped = grid.warp_by_scalar()
    # plotter.add_mesh(warped)
    plotter.show()
except ModuleNotFoundError:
    print("'pyvista' is required to visualise the solution")
    print("Install 'pyvista' with pip: 'python3 -m pip install pyvista'")

In [ ]:
uh.x